# SNSの構成

## システム概要
- テキストや写真・動画によりユーザの非同期コミュニケーションを実現するSNS
- コンテンツ投稿（動画・写真・音声等）、リアルタイムチャット、検索、プッシュ通知、通話、ライブストリーミングなど主要なSNS機能を全て備える

## 構成図
<iframe width="100%" height="800" src="https://miro.com/app/live-embed/uXjVK3v8SZ0=/?moveToViewport=7638,-980,4619,3724&embedId=90811350354" frameborder="0" scrolling="no" allow="fullscreen; clipboard-read; clipboard-write" allowfullscreen></iframe>

## 構成要素

```mermaid
classDiagram
    class CDN {
        役割：コンテンツ配信とキャッシュ
    }
    
    class Gateway {
        役割：ルーティング制御と受付
    }
    
    class Repository {
        役割：データ永続化管理
    }
    class Context {
        役割：コンテキスト情報の保持
    }
    class Text {
        役割：テキストデータの管理
    }
    class Media {
        役割：メディアファイルの管理
    }
    Repository --> Context
    Repository --> Text
    Repository --> Media
    
    class API {
        役割：ビジネスロジック制御
    }
    class Stateful {
        役割：状態を持つAPI処理
    }
    class Stateless {
        役割：状態を持たないAPI処理
    }
    class Command {
        役割：データ更新操作
    }
    class Query {
        役割：データ参照操作
    }
    API --> Stateful
    API --> Stateless
    Stateless --> Command
    Stateless --> Query
    
    class UI {
        役割：ユーザーインターフェース提供
    }
    
    class Auth {
        役割：認証・認可制御
    }
    
    class Observability {
        役割：システム監視・可観測性確保
    }
    
    class Secrets {
        役割：機密情報管理
    }
```

## 考慮事項

| 分類 | 対策 | 実装 |
|------|------|----------|
| 可用性 | マルチAZ、マルチリージョン、レプリケーション、キャッシング、フェイルオーバー ||
| スケーラビリティ | IaC/Git/CD/CI、Clean Architecture、イベント駆動、オートスケーリング ||
| セキュリティ | 認証認可、シークレット管理、暗号化、監視 ||
